# How to Separate Calexp Catalog Sources into Different Magnitude Categories.

This notebook illustrates how to separate calexp catalog sources into three catagories: saturation stars, super bright stars, and less bright stars.

In [2]:
import pandas as pd
import numpy as np

from astropy.table import Table
from lsst.daf.persistence import Butler
from lsst.afw.image import PhotoCalib

## 1. Get calexp sources

To begin with, we need to specify the `visit`, `detector`, and `filter` information of the calexp image. We also need to specify the `repo` of the calexp image. Next we do calibration to the source catalog. Relevent columns are saved in `calexp_source.csv`.

In [6]:
# visit, detector and filter information
visit, detector, filt = 400390, 157, 'g'
calexp_id = {'visit': visit, 'detector': detector, 'filter': filt}

# calexp repo
calexp_repo = '/global/cscratch1/sd/desc/DC2/data/Run2.1i/rerun/calexp-v1'
calexp_butler = Butler(calexp_repo)

# Get calexp catalog
calexp_catalog = calexp_butler.get('src', calexp_id)

# Calibration
calexp_Calib = calexp_butler.get('calexp_photoCalib',  calexp_id)
calexp_mag = calexp_Calib.instFluxToMagnitude(calexp_catalog, 'base_PsfFlux')

# Get relevant columns
calexp_astropy = calexp_catalog.asAstropy()
calexp_astropy2 =  calexp_astropy[['coord_ra', 'coord_dec', \
'base_FPPosition_x', 'base_FPPosition_y', \
'base_NaiveCentroid_x', 'base_NaiveCentroid_y', \
'deblend_psfCenter_x', 'deblend_psfCenter_y', \
'base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_saturatedCenter' \
]] 
calexp_astropy2['calexp_magnitude'] = calexp_mag[:, 0]
calexp_astropy2['calexp_magnitude_err'] = calexp_mag[:, 1]

# Save results
calexp_astropy2.write('calexp_source.csv', format='ascii.csv')

# 2. pick up stars

Here we separate stars into three catagories: `saturation stars`, `super bright stars`, and `less bright stars`.

In [8]:
# Read source table
calexp_catalog = Table.read('./calexp_source.csv', format='ascii.csv')
# Get the floor of the minimum magnitude
min_mag = np.floor(np.min(calexp_catalog['calexp_magnitude'].data))
print(min_mag)

16.0


## a). saturation stars
#### saturatedCenter = True and saturated = True

In [11]:
row_num = []
for num, src in enumerate(calexp_catalog):
    if src['base_PixelFlags_flag_saturatedCenter'] == 'True' \
    and src['base_PixelFlags_flag_saturated'] == 'True':
        row_num.append(num)
satur_center = calexp_catalog[row_num]
satur_center['coord_ra'] = np.rad2deg(satur_center ['coord_ra'])
satur_center['coord_dec'] = np.rad2deg(satur_center ['coord_dec'])
satur_center_coord = satur_center[['coord_ra', 'coord_dec']]
satur_center_coord.write('./saturation_stars.csv', format='ascii.csv')

In [12]:
print(len(satur_center_coord))

21


## b). super bright stars

In [13]:
print(min_mag)

16.0


We choose stars with magnitude between ming_mag(16) and 18 as super bright stars.

In [14]:
row_num = []
for num, src in enumerate(calexp_catalog):
    if src['base_PixelFlags_flag_saturatedCenter'] == 'False' \
    and src['base_PixelFlags_flag_saturated'] == 'False' \
    and src['calexp_magnitude'] >= min_mag \
    and src['calexp_magnitude'] < 18:
        row_num.append(num)
super_bright = calexp_catalog[row_num]
super_bright['coord_ra'] = np.rad2deg(super_bright['coord_ra'])
super_bright['coord_dec'] = np.rad2deg(super_bright['coord_dec'])

super_bright_coord = super_bright[['coord_ra', 'coord_dec']]
super_bright_coord.write('./super_bright_stars.csv', format='ascii.csv')

In [15]:
print(len(super_bright_coord))

24


## c). less bright stars

We choose stars with magnitude between 18 and 20 as less bright stars.

In [16]:
row_num = []
for num, src in enumerate(calexp_catalog):
    if src['base_PixelFlags_flag_saturatedCenter'] == 'False' \
    and src['base_PixelFlags_flag_saturated'] == 'False' \
    and src['calexp_magnitude'] >= 18 \
    and src['calexp_magnitude'] < 20:
        row_num.append(num)
less_bright = calexp_catalog[row_num]
less_bright['coord_ra'] = np.rad2deg(less_bright['coord_ra'])
less_bright['coord_dec'] = np.rad2deg(less_bright['coord_dec'])
less_bright_coord = less_bright[['coord_ra', 'coord_dec']]
less_bright_coord.write('./less_bright_stars.csv', format='ascii.csv')

In [17]:
print(len(less_bright_coord))

57
